# Getting started with Cross-region Inference in Amazon Bedrock
Inference Profiles via Amazon Bedrock allow you to perform cross-region inference without the need to setup anything in a fully-managed, secure and private manner. Inference profiles is a powerful feature built on Amazon Bedrock, offering generative AI application builders a seamless solution for managing traffic bursts and ensuring optimal availability and performance. By leveraging this feature, builders no longer have to spend time and effort building complex resiliency structure. Instead, inference profiles intelligently routes traffic across multiple opted-in regions, automatically adapting to peak utilization surges. Any inferences carried out through inference profiles will dynamically utilize on-demand capacity available from any of the configured regions, maximizing availability and throughput.

### Key Features & Benefits:
Some of the key features include:

* Access to capacity in different regions allowing generative AI workloads to scale with demand.
* Access to region-agnostic models to achieve higher throughput.
* Become resilient to any traffic bursts.
* Ability to select between the pre-defined region sets suiting application needs.
* Compatible with existing APIs.
* No additional cost.
* Same model pricing as the primary region.

The end-user experience is not impacted as the model under inference profiles remain the same, and builder can focus on writing logic for a differentiated application. 

Bedrock is the easiest way to build and scale generative AI applications. Cross region inference further enhances the usability.

In [ ]:
# Start by installing the dependencies to ensure we have a recent version
!pip install --quiet --upgrade --force-reinstall boto3 botocore awscli
import boto3
print(boto3.__version__)

🔴 IMPORTANT❗🔴

*Note:* <span style="color:red">This notebook sample uses `us-east-1` region and the inference profiles available there as example.
Change the `region_name` and inference profile ids in the cells below depending on which region you are in and which inference profiles are available.</span>

In [ ]:
# Make sure you have AWS credentials or AWS profile setup before running this cell
region_name = 'us-east-1'
bedrock_client = boto3.client('bedrock', region_name=region_name)
bedrock_runtime = boto3.client('bedrock-runtime', region_name=region_name)

### Understand Inference Profiles
The inference profiles feature comes with 2 additional APIs in the Bedrock client SDK:

* `list_inference_profiles()` -> This API tells you all the models that are configured behind Inference Profiles for you.
* `get_inference_profile(inferenceProfileIdentifier)` -> This API give you specific details of a certain Inference Profile.

In [ ]:
bedrock_client.list_inference_profiles()['inferenceProfileSummaries']

It is important to note that a Inference Profile can exist in 2 forms:

**Foundation model in primary region**

In this mode a Inference Profile is only configured for a model which exists in the primary region. For such model(s) a failover mechanism would exist allowing requests to be re-routed to secondary regions configured in Inference Profile. By default the request will go to primary region resources and only if the region is busy or you hit your quota limit, the request is routed to another region. In order to determine, which region the request should go to Amazon Bedrock intelligently checks in real-time which region has redundant capacity available. The on-demand principle still applies if none of the region has capacity to handle the request, then it will be throttled.

In this mode, If the primary region doesn't have a certain model available, you will not be able to access it in a secondary region via cross-region inference feature.

**Available via Inference Profiles**

Select list of models are made available via Inference Profile, where Amazon Bedrock abstracts away the regional details and manages the hosting and inference routing automatically. Such foundation models then exist across the pre-defined region sets and the builder can build applications agnostic of setting up a region. This allows reliable throughput, access to leading foundation models as well as scalability in terms of throughput.

In [ ]:
bedrock_client.get_inference_profile(
    inferenceProfileIdentifier='us.anthropic.claude-3-5-sonnet-20240620-v1:0'
)

With the `get_inference_profile` API you can observe the `status` if a Inference Profile is `ACTIVE` or not and also which regions are configured for inference routing.

### Use Inference Profiles
An Inference Profile is used in the same way as a foundation model using the `modelId` or `arn` of the model. Inference profile also comes with it's own id and arn, where the difference is in the prefix. For the inference profile you can expect a regional prefix such as `us.` or `eu.` behind the model id for it to be recognized as an inference profile. Also in the `arn`, you can find the difference from `:<region>::foundation-model/<model-id>` to `:<region>::inference-profile/<region-set-prefix>.<model-id>`

You can use both the `arn` and the `modelId` with `Converse` API whereas only the `modelId` with `InvokeModel` API

#### Converse API
Amazon Bedrock now supports a unified messaging API for seamless application building experience. Read about all the models supported via this API [here](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html#conversation-inference-supported-models-features).

Let's send a request to both the foundation model as well as the Inference Profile to observe change.

In [ ]:
from time import time
system_prompt = "You are an expert on AWS services and always provide correct and concise answers."
input_message = "Should I be storing documents in Amazon S3 or EFS for cost effective applications?"
modelId = ('Foundation Model', 'anthropic.claude-3-haiku-20240307-v1:0')
inferenceProfileId = ('Inference Profile', 'us.anthropic.claude-3-haiku-20240307-v1:0')

for inference_type, id in [modelId, inferenceProfileId]:
    start = time()
    response = bedrock_runtime.converse(
        modelId=id,
        system=[{"text": system_prompt}],
        messages=[{
            "role": "user",
            "content": [{"text": input_message}]
        }]
    )
    end = time()
    print(f"::{inference_type}::Response time: {int(end-start)} second(s)")
    print(f"::{inference_type}::Response output: {response['output']['message']['content']}")

You can observe that using the same API and only the change in model id you can expect similar behavior.

It is also possible to use `inferenceProfileArn` with the `Converse` API:

In [ ]:
inferenceProfileArn = 'arn:aws:bedrock:us-east-1::inference-profile/us.anthropic.claude-3-sonnet-20240229-v1:0'

response = bedrock_runtime.converse(
    modelId=inferenceProfileArn,
    system=[{"text": system_prompt}],
    messages=[{
        "role": "user",
        "content": [{"text": input_message}]
    }]
)
print(response['output']['message']['content'][0]['text'])

#### InvokeModel API
Most of the generative AI applications in production are already built on top of `InvokeModel` API, even the third-party tools also use this API for using the models. The cross-region inference feature is also compatible with this API. Where the Converse API only supports select models, all models available in Amazon Bedrock can leverage InvokeModel API.

Let's send a request via both the foundation model id as well as the inference profile id to observe change via this API.

In [ ]:
import json
body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1024,
    "temperature": 0.1,
    "top_p": 0.9,
    "system": system_prompt,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"{input_message}",
                }
            ]
        }
    ]
})
accept = 'application/json'
contentType = 'application/json'
modelId = ('Foundation Model', 'anthropic.claude-3-sonnet-20240229-v1:0')
inferenceProfileId = ('Inference Profile', 'us.anthropic.claude-3-sonnet-20240229-v1:0')
for inference_type, id in [modelId, inferenceProfileId]:
    start = time()
    response = bedrock_runtime.invoke_model(body=body, modelId=id, accept=accept, contentType=contentType)
    end = time()
    response_body = json.loads(response.get('body').read())
    print(f"::{inference_type}::Response time: {int(end-start)} second(s)")
    print(f"::{inference_type}::Response output: {response_body['content'][0]['text']}")

#### LangChain
Below you can find integration on how to use the new Inference Profiles feature with one of the popular open-source frameworks [LangChain](https://python.langchain.com/v0.2/docs/integrations/platforms/aws/).

In [ ]:
!pip install --quiet langchain_aws langchain_community

[LangChain](https://python.langchain.com/v0.2/docs/introduction/) with the help of integrations implements [`langchain-aws`](https://github.com/langchain-ai/langchain-aws) which provides support  `Converse` API via [`ChatBedrockConverse`](https://python.langchain.com/v0.2/docs/integrations/chat/bedrock/#beta-bedrock-converse-api). This allows you to use the latest models such as Anthropic Claude 3 Sonnet via this implementation. Below you can see an example of that.

In [ ]:
from langchain_aws import ChatBedrockConverse

messages = [
    (
        "system",
        "You are a helpful assistant that shapes sentences into poetic form. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]

llm = ChatBedrockConverse(
    model='us.anthropic.claude-3-sonnet-20240229-v1:0',
    temperature=0,
    max_tokens=None,
    client=bedrock_runtime,
)

print(llm.invoke(messages).content)

## Conclusion
Inference Profiles provide the ability to manage bursts and spiky traffic patterns across a variety of generative AI workloads and disparate request shapes.With this feature you can easily scale your workloads in production without the need of heavy-lifting, lengthy migrations and overhead of infrastructure management. Amazon Bedrock handles the routing securely, reliably and in a transparent manner while giving you the control you need.

### Key considerations
While building or migrating applications to use Inference Profiles, it is important to keep in mind the following:
- **Latency** - If your application is latency sensitive, it is advised to properly test the use of Inference Profiles prior to fully relying on it since the routing to different regions could lead to higher latency numbers thus impacting your application behavior.
- **Compliance** - Inference Profiles come with pre-defined region sets, if these sets contain a region where you can't operate or goes against your policy, then it is advised not to use Inference Profiles, instead utilize Foundation Models directly.
- **Determinism** - If you need to have control over where your requests are (or will be) re-routed (other than primary region), it is better to consider just rely on Foundation Model directly. Also the Inference Profile exclusive models should be opted for carefully.
- **Variety** - Inference Profiles do not provide access to mulitple different models from different regions, it either acts as a failover mechanism for models in primary region or provides Inference Profile exclusive models where the construct of a region is abstracted away. If your business demands to span across variety of foundation models from mulitple regions, it is wise to consider building your own architecture via VPC Peering/Transit Gateway or other architectural patterns.